In [1]:
# Use low power tech to sleep until a rocker switch 
# flips and use its setting to send mqtt controls out


No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands

In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB5 --baud=115200 
Ready.


In [7]:
%fetchfile --print config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.100.1
pinyellow    4
pingreen     21
pinred       18
pinrocker    2
boardname    esp32flip
deepsleepsec 15


In [2]:
# This requires you to checkout https://github.com/micropython/micropython-lib and avoids the upip
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio/uasyncio/__init__.py lib/uasyncio/
%sendtofile --quiet --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.core/uasyncio/core.py lib/uasyncio/
%sendtofile --quiet --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.queues/uasyncio/queues.py lib/uasyncio/
%sendtofile --quiet --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.synchro/uasyncio/synchro.py lib/uasyncio/

# This requires you to checkout https://github.com/peterhinch/micropython-mqtt/tree/master/mqtt_as
%sendtofile --quiet --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py

%sendtofile --quiet --source utils.py

Sent 258 lines (8400 bytes) to lib/uasyncio/__init__.py.
Sent 315 lines (9509 bytes) to lib/uasyncio/core.py.
Sent 94 lines (2691 bytes) to lib/uasyncio/queues.py.
Sent 28 lines (846 bytes) to lib/uasyncio/synchro.py.
Sent 660 lines (23451 bytes) to mqtt_as.py.
Sent 31 lines (960 bytes) to utils.py.


In [3]:
%sendtofile config.txt

wifiname     BV6000
wifipassword bubblino
mqttbroker   192.168.43.1
pinyellow    4
pingreen     21
pinred       18
pinrocker    2
boardname    esp32flip
deepsleepsec 600


Sent 9 lines (170 bytes) to config.txt.


In [9]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.100.1
pinyellow    4
pingreen     21
pinred       18
pinrocker    2
boardname    esp32flip
deepsleepsec 600


Sent 9 lines (177 bytes) to config.txt.


In [22]:
%serialconnect

Connecting to --port=/dev/ttyUSB5 --baud=115200 
Ready.


In [23]:

%sendtofile main.py

import machine, time, esp32
rtc = machine.RTC()
def ftime():
    d = rtc.datetime()
    return time.mktime((d[0], d[1], d[2], d[4], d[5], d[6], 0, 0)) + (d[7]//1000)/1000

timestartup = ftime()
timeconnected = 0

resetcause = ["0", "PWRON", "HARD", "WDT", "DEEPSLEEP", "SOFT"][machine.reset_cause()]+"_RESET"
wakereason = ["0", "1", "EXT0", "EXT1", "TIMER", "TOUCHPAD", "ULP"][machine.wake_reason()]+"_WAKE"

print("Startup secs=%.3f: %s, %s" % (timestartup, resetcause, wakereason))
rtcmemory = rtc.memory()
print("rtcmemory:", rtcmemory, time.ticks_ms())

from machine import Pin, PWM
fconfig = dict(x.split()  for x in open("config.txt"))

pinyellow = Pin(int(fconfig["pinyellow"]), Pin.OUT)
pingreen = PWM(Pin(int(fconfig["pingreen"])), duty=0)
pinred = PWM(Pin(int(fconfig["pinred"])), duty=0)
pinrocker = Pin(int(fconfig["pinrocker"]), Pin.IN, Pin.PULL_DOWN)

if not rtcmemory:
    for i in range(1, 5):
        pinyellow.value(i%2)
        time.sleep_ms(50)

rockerpwmfreq = const(60)
rockerflashfreq = const(4)
rockerfduration_toflash = const(4000)
rockerfduration_tosleep = const(5000)
connectbylimit = const(10000)

rockervalueprev = 0  # if left as -1 then no change has happened
rockervalue = -1
rockertime = 0
rockerfduration = 0

import uasyncio as asyncio

async def ledrockertask():
    global rockervalue, rockervalueprev, rockertime, rockerfduration
    while True:
        rockervalueprev, rockervalue = rockervalue, pinrocker.value()
        if rockervalue != rockervalueprev:
            pingreen.freq(rockerpwmfreq)
            rockertime = time.ticks_ms()
            rockerfduration = 0
            if rockervalueprev == -1 and machine.wake_reason() == machine.TIMER_WAKE:
                rockerfduration = rockerfduration_toflash
            if rockervalue == 1:
                pingreen.duty(20)
                pinred.duty(0)
            else:
                pinred.duty(20)
                pingreen.duty(0)
        else:
            rockerfduration = time.ticks_ms() - rockertime
            if rockerfduration > rockerfduration_toflash and pingreen.freq() != rockerflashfreq:
                pingreen.freq(rockerflashfreq)
        await asyncio.sleep_ms(100)

async def resleeptask():
    pinyellow.value(1)
    await asyncio.sleep_ms(50)
    pinyellow.value(0)
    t0 = time.ticks_ms()
    while True:
        if rockerfduration >= rockerfduration_tosleep:
            pinyellow.value(1)
            dt = time.ticks_ms() - t0
            if client.isconnected() or dt > connectbylimit:
                break
            await asyncio.sleep_ms(50)
            pinyellow.value(0)
        await asyncio.sleep_ms(300)
        
    msg = "%d %s %.3f" % (rockervalue, rtcmemory.decode(), timestartup)
    if client.isconnected():
        await client.publish(boardname+b"/direction", msg)
    esp32.wake_on_ext0(pinrocker, (esp32.WAKEUP_ANY_HIGH if rockervalue == 0 else esp32.WAKEUP_ALL_LOW))
    timeatsleep = ftime()
    print("time at sleep:", timeatsleep)
    rtc.memory("%d %.3f %.3f %.3f %s %s" % (rockervalue, timestartup, timeconnected, timeatsleep, resetcause, wakereason))
    client._sta_if.active(False)   # bug number ...
    machine.deepsleep(int(fconfig["deepsleepsec"])*1000)
        

from mqtt_as import config, MQTTClient
config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['mqttchannel'] = fconfig["boardname"]

if not rtcmemory:
    for i in range(1, 5):
        pinred.duty((i%2)*20)
        time.sleep_ms(50)
client = MQTTClient(config)   # this is causing the brownout
boardname = fconfig["boardname"].encode()
if not rtcmemory:
    for i in range(1, 5):
        pinred.duty((i%2)*20)
        time.sleep_ms(50)

async def mqtttask():
    global timeconnected
    try:
        await client.connect()
        timeconnected = ftime()
        print("time to connect:", timeconnected - timestartup)
    except OSError as e:
        print("client connect error", [e])

aloop = asyncio.get_event_loop()
aloop.create_task(ledrockertask())
aloop.create_task(resleeptask())
aloop.create_task(mqtttask())

if not rtcmemory:
    for i in range(1, 5):
        pinred.duty((i%2)*20)
        time.sleep_ms(50)

aloop.run_forever()


Sent 127 lines (4243 bytes) to main.py.


In [5]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB6 --baud=115200 
Ready.


In [7]:
print(client.__dict__)

{'_sta_if': <WLAN>, '_has_connected': False, '_ssl': False, '_keepalive': 60, '_clean_init': True, '_ssl_params': {}, '_connect_handler': <generator>, 'lock': <Lock object at 3ffc5c30>, '_wifi_handler': <generator>, '_ping_interval': 15000, '_sock': None, '_lw_topic': False, 'newpid': <generator object 'pid_gen' at 3ffc5c90>, '_clean': True, 'last_rx': 70230, '_cb': <function <lambda> at 0x3ffc4150>, 'server': '192.168.43.1', '_in_connect': False, 'port': 1883, '_response_time': 10000, '_isconnected': False, 'rcv_pids': set(), '_pswd': '', '_ssid': 'BV6000', '_wifi_pw': 'bubblino', '_client_id': b'240ac400412e', '_max_repubs': 4, '_user': ''}


In [10]:
print(dir(client._sta_if))

['__class__', 'active', 'config', 'connect', 'disconnect', 'ifconfig', 'isconnected', 'scan', 'status']


In [9]:
async def hi(t):
    await asyncio.sleep(t)
    print("go error", t)
    raise OSError()
    
aloop = asyncio.get_event_loop()
aloop.create_task(hi(2))
aloop.create_task(hi(3))
aloop.run_forever()


......

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 7, in <module>
  File "/lib/uasyncio/core.py", line 173, in run_forever
  File "/lib/uasyncio/__init__.py", line 69, in wait
KeyboardInterrupt: 


In [3]:
# Below is code to test whether the timeouts work

In [37]:
%sendtofile main.py

import uasyncio as asyncio
fconfig = dict(x.split()  for x in open("config.txt"))

from mqtt_as import config, MQTTClient
import mqtt_as
mqtt_as.LOBO = True

config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['mqttchannel'] = fconfig["boardname"]
client = MQTTClient(config)
boardname = fconfig["boardname"].encode()

async def mqtttask():
    print("enter connect")
    try:
        await client.connect()
    except OSError as e:
        print("client connect error", [e])

aloop = asyncio.get_event_loop()
aloop.run_until_complete(mqtttask())


Sent 23 lines (624 bytes) to main.py.


In [20]:
%serialconnect

Connecting to --port=/dev/ttyUSB5 --baud=115200 
Ready.


In [17]:
import network
w = network.WLAN()
w.active(1)
w.scan()



Brownout detector was triggered

ets Jun  8 2016 00:22:57

rst:0xc (SW_CPU_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:2
load:0x3fff0018,len:4
load:0x3fff001c,len:4928
ho 0 tail 12 room 4
load:0x40078000,len:9656
ho 0 tail 12 room 4
load:0x40080400,len:6252
entry 0x400806f4
memory: b''
0142535625139749352236144856896779769164275836317692621118686877675178774936725778853658346511277798
0142535625139749352236144856896779769164275836317692621118686877675178774936725778853658346511277798
0142535625139749352236144856896779769164275836317692621118686877675178774936725778853658346511277798
0142535625139749352236144856896779769164275836317692621118686877675178774936725778853658346511277798
0142535625139749352236144856896779769164275836317692621118686877675178774936725778853658346511277798
ets Jun  8 2016 00:22:57

rst:0x5 (DEEPSLEEP_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
configsip: 

In [21]:
%sendtofile main.py

import machine, time, math
r = machine.RTC()
print("memory:", r.memory())
if r.memory():
    r.memory("")
else:
    r.memory("hi there"*8)
    
time.sleep_ms(1000)

import network
w = network.WLAN()
w.active(1)
w.scan()

for i in range(5):
    time.sleep_ms(500)
    print("".join(str(math.sin(j))[-1]  for j in range(100)))
    
pinrocker = machine.Pin(2, machine.Pin.IN, machine.Pin.PULL_DOWN)
rockervalue = 0
import esp32
#esp32.wake_on_ext0(pinrocker, (esp32.WAKEUP_ANY_HIGH if rockervalue == 0 else esp32.WAKEUP_ALL_LOW))

#w.active(0)
machine.deepsleep(3000)



Sent 27 lines (566 bytes) to main.py.


In [6]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.11-406-g4a6974bea on 2019-10-06; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f8ad8ded208, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [ ]:
import machine, network, esp32
w = network.WLAN()
w.active(1)
w.scan()
p = machine.Pin(2, machine.Pin.IN, machine.Pin.PULL_DOWN)
esp32.wake_on_ext0(p, esp32.WAKEUP_ANY_HIGH)

#w.active(0)
machine.deepsleep(3000)


#0.006mA  
#1.4mA

